In [1]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
#from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

In [2]:
def historico_partidas(tabela_url, paginas):
  todas_partidas = []
  data = requests.get(tabela_url) # Pega o html (em forma de string) da pagina 'Tabela do Brasileirao'
  soup = BeautifulSoup(data.text) # Converte o String em html
  tabela_brasileirao = soup.select('table.stats_table')[0] # Seleciona dentro do HTML a tabela em si
  links_times = tabela_brasileirao.find_all('a') # Procura a Anchor que contem todos os links do time
  links_times = [link.get("href") for link in links_times] # Pega só os links em si
  links_times = [link for link in links_times if paginas[0] in link] # Escolhe o link que leva para a pagina de Stats do time
  times_urls = [f"https://fbref.com{link}" for link in links_times] # Transforma a string em URL

  # Para cada time faça:
  ## 1- Pega o nome do time
  ## 2- Pega as informações das abas (em ordem):
  ### Scores & Fixtures, Shooting, Goalkeeping, Passing, Pass Types, Goal and Shot Creation, Defensive Actions, Possession, Miscellaneous Stats
  ## 3- Transforma a tabela que esta em HTML em Dataframe
  for time_url in times_urls:
    nome_time = time_url.split('/')[-1].replace("-Stats", "")

    data = requests.get(time_url) # Pega o html (em forma de string) da pagina do time
    ## Scores & Fixtures
    partidas = pd.read_html(data.text, match= 'Scores & Fixtures')[0].drop(
      ['Captain', 'Formation', 'Referee', 'Match Report'], axis= 'columns') # Transforma em dataframe o historico de partidas
    ## Shooting
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and paginas[1] in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match= "Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    ## Goalkeeping
    ## Passing
    ## Pass Types
    ## Goal and Shot Creation
    ## Defensive Actions
    ## Possession
    ## Miscellaneous Stats

    # Merge das abas
    try:
      time_df = partidas.merge(
        shooting[['Date', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'FK', 'PK', 'PKatt', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG']], 
        #goalkeeping[[]], 
        #passing[[]], 
        #pass_types[[]], 
        #goal_shotcreation[[]], 
        #defensive[[]], 
        #possession[[]], 
        #miscellaneous[[]], 
        on= "Date")
    except ValueError:
      continue
    
    time_df = time_df[time_df["Comp"] == "Série A"] # Filtra o brasileirão
    time_df['Equipe'] = nome_time
    todas_partidas.append(time_df)
    time.sleep(5)

    historico = pd.concat(todas_partidas)
    return historico

tabela = "https://fbref.com/en/comps/24/Serie-A-Stats"
abas = ['/squads/', 'all_comps/shooting/']
historico = historico_partidas(tabela, abas)
historico

IndexError: list index out of range